In [25]:
#!cp 1-gp.py 2-gp.py
!chmod +x *.py

In [29]:
#!/usr/bin/env python3
""" Hyperparameter Tuning """
import numpy as np


class GaussianProcess():
    """ noiseless 1D Gaussian process """

    def __init__(self, X_init, Y_init, l=1, sigma_f=1):
        """
        *************************************************
        ***************** constructor *******************
        *************************************************
        @X_init: is a numpy.ndarray of shape (t, 1)
                 representing the inputs already sampled
                 with the black-box function
        @Y_init: is a numpy.ndarray of shape (t, 1)
                 representing the outputs of the black-box
                 function for each input in X_init
        @t is: the number of initial samples
        @l is: the length parameter for the kernel
        @sigma_f: is the standard deviation given to the
                  output of the black-box function
        """
        self.X = X_init
        self.Y = Y_init
        self.l = l
        self.sigma_f = sigma_f
        self.K = self.kernel(X_init, X_init)

    def kernel(self, X1, X2):
        """
        calculates the covariance kernel
        matrix between two matrices using
        use the Radial Basis Function (RBF)

        @X1: is a numpy.ndarray of shape (m, 1)
        @X2: is a numpy.ndarray of shape (n, 1)

        Returns: the covariance kernel matrix as
                 a numpy.ndarray of shape (m, n)
        """
        return self.sigma_f**2 * np.exp(pow(X1 - X2.T, 2)/-2/self.l**2)

    def predict(self, X_s):
        """
        predicts the mean and standard deviation
        of points in a Gaussian process

        @X_s: is a numpy.ndarray of shape (s, 1) containing all of
              the points whose mean and standard deviation should be calculated
        @s: is the number of sample points
        Returns: mu, sigma
                 mu: is a numpy.ndarray of shape (s,) containing the mean
                     for each point in X_s, respectively
                 sigma: is a numpy.ndarray of shape (s,) containing the variance
                        for each point in X_s, respectively
        """
        K_s = self.kernel(X_s, self.X)
        K_inv = np.linalg.inv(self.K)
        mu = np.matmul(np.matmul(K_s, K_inv), self.Y)[:, 0]
        K_s2 = self.kernel(X_s, X_s)
        sigma = K_s2 - np.matmul(np.matmul(K_s, K_inv), K_s.T)
        return mu, np.diagonal(sigma)

    def update(self, X_new, Y_new):
        """
        updates a Gaussian Process
        => Updates the public instance attributes X, Y, and K

        X_new: is a numpy.ndarray of shape (1,)
               that represents the new sample point
        Y_new: is a numpy.ndarray of shape (1,)
               that represents the new sample function value
        """
        self.X = np.append(self.X, X_new[:, None], axis=0)
        self.Y = np.append(self.Y, Y_new[:, None], axis=0)
        self.K = self.kernel(self.X, self.X)
GP = GaussianProcess

In [23]:
#!/usr/bin/env python3

#GP = __import__('0-gp').GaussianProcess
import numpy as np


def f(x):
    """our 'black box' function"""
    return np.sin(5*x) + 2*np.sin(-2*x)

if __name__ == '__main__':
    np.random.seed(0)
    X_init = np.random.uniform(-np.pi, 2*np.pi, (2, 1))
    Y_init = f(X_init)

    gp = GaussianProcess(X_init, Y_init, l=0.6, sigma_f=2)
    print(gp.X is X_init)
    print(gp.Y is Y_init)
    print(gp.l)
    print(gp.sigma_f)
    print(gp.K.shape, gp.K)
    print(np.allclose(gp.kernel(X_init, X_init), gp.K))

True
True
0.6
2
(2, 2) [[4.         0.13150595]
 [0.13150595 4.        ]]
True


In [24]:
#!/usr/bin/env python3


import numpy as np


def f(x):
    """our 'black box' function"""
    return np.sin(5*x) + 2*np.sin(-2*x)

if __name__ == '__main__':
    np.random.seed(0)
    X_init = np.random.uniform(-np.pi, 2*np.pi, (2, 1))
    Y_init = f(X_init)

    gp = GP(X_init, Y_init, l=0.6, sigma_f=2)
    X_s = np.random.uniform(-np.pi, 2*np.pi, (10, 1))
    mu, sig = gp.predict(X_s)
    print(mu.shape, mu)
    print(sig.shape, sig)

(10,) [ 0.20148983  0.93469135  0.14512328 -0.99831012  0.21779183 -0.05063668
 -0.00116747  0.03434981 -1.15092063  0.9221554 ]
(10,) [1.90890408 0.01512125 3.91606789 2.42958747 3.81083574 3.99817545
 3.99999903 3.9953012  3.05639472 0.37179608]


In [30]:
#!/usr/bin/env python3

#GP = __import__('2-gp').GaussianProcess
import numpy as np


def f(x):
    """our 'black box' function"""
    return np.sin(5*x) + 2*np.sin(-2*x)

if __name__ == '__main__':
    np.random.seed(0)
    X_init = np.random.uniform(-np.pi, 2*np.pi, (2, 1))
    Y_init = f(X_init)

    gp = GP(X_init, Y_init, l=0.6, sigma_f=2)
    X_new = np.random.uniform(-np.pi, 2*np.pi, 1)
    print('X_new:', X_new)
    Y_new = f(X_new)
    print('Y_new:', Y_new)
    gp.update(X_new, Y_new)
    print(gp.X.shape, gp.X)
    print(gp.Y.shape, gp.Y)
    print(gp.K.shape, gp.K)

X_new: [2.53931833]
Y_new: [1.99720866]
(3, 1) [[2.03085276]
 [3.59890832]
 [2.53931833]]
(3, 1) [[ 0.92485357]
 [-2.33925576]
 [ 1.99720866]]
(3, 3) [[4.         0.13150595 2.79327536]
 [0.13150595 4.         0.84109203]
 [2.79327536 0.84109203 4.        ]]
